In [1]:
# --- 1. KẾT NỐI GOOGLE DRIVE (Để lấy dữ liệu) ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!pip install pyspark

In [3]:
# LOAD DỮ LIỆU ĐẦU VÀO (DATA LOADING)

from pyspark.sql import SparkSession

# 1. Khởi tạo Spark
spark = SparkSession.builder.appName("DS_Model").getOrCreate()

# 2. Định nghĩa đường dẫn chung
base_path = "/content/drive/MyDrive/dulieulon/BTL"
parquet_path = f"{base_path}/processed_house_data.parquet"
csv_path = f"{base_path}/zipcode_coords.csv"

# ------------------------------------------------------------------------------
# PHẦN A: ĐỌC DỮ LIỆU CHÍNH (PARQUET)
# ------------------------------------------------------------------------------
try:
    df_house = spark.read.parquet(parquet_path)
    print(f"✅ [1/2] Tải thành công Dữ liệu Nhà (Parquet)!")
    print(f"   -> Tổng số dòng: {df_house.count():,}")
    df_house.printSchema()
except Exception as e:
    print(f"❌ Lỗi đọc Parquet: {e}")

print("-" * 30)

# ------------------------------------------------------------------------------
# PHẦN B: ĐỌC DỮ LIỆU ZIPCODE (CSV)
# ------------------------------------------------------------------------------
try:
    # Lưu ý: Với CSV phải có header=True để nó hiểu dòng đầu là tên cột
    df_zip = spark.read.csv(csv_path, header=True, inferSchema=True)
    print(f"✅ [2/2] Tải thành công Dữ liệu Zipcode (CSV)!")
    print(f"   -> Tổng số Zipcode: {df_zip.count()}")

    # In thử vài dòng xem tọa độ đúng chưa
    print("   -> Mẫu dữ liệu:")
    df_zip.show(3)
except Exception as e:
    print(f"❌ Lỗi đọc CSV: {e}")

print("=" * 50)
print(" DỮ LIỆU ĐÃ SẴN SÀNG ĐỂ TRAIN MODEL!")

✅ [1/2] Tải thành công Dữ liệu Nhà (Parquet)!
   -> Tổng số dòng: 21,596
root
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: double (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)
 |-- sell_month: integer (nullable = true)

------------------------------
✅ [2/2] Tải thành công Dữ liệu Zipcode (CSV)!
   -> Tổng số Zipcode: 70

# Chuẩn bị dữ liệu

In [4]:
# --- CELL 1: SETUP & DATA PREP ---
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

# 1. Định nghĩa các biến đầu vào (Features)
feature_cols = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
    'waterfront', 'view', 'condition', 'grade',
    'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
    'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sell_month'
]

# 2. Vector Assembler (Gom cột)
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# 3. Chia tập dữ liệu (80% Train - 20% Test)
train_data, test_data = df_house.randomSplit([0.8, 0.2], seed=42)

print(f"✅ Đã chia dữ liệu xong!")
print(f"   - Tập huấn luyện (Train): {train_data.count():,} dòng")
print(f"   - Tập kiểm tra (Test): {test_data.count():,} dòng")

✅ Đã chia dữ liệu xong!
   - Tập huấn luyện (Train): 17,334 dòng
   - Tập kiểm tra (Test): 4,262 dòng


# Chạy LINEAR REGRESSION
Áp dụng Grid Search để tìm tham số phạt L1/L2 tối ưu

In [13]:
print("🚀 ĐANG CHẠY LINEAR REGRESSION (ELASTIC NET)...")
print("   (Đang dò tìm tham số phạt regParam và elasticNetParam tốt nhất...)")

# 1. Khởi tạo
lr = LinearRegression(featuresCol="features", labelCol="price")
lr_pipeline = Pipeline(stages=[assembler, lr])

# 2. Tạo lưới tham số (Grid Search)
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, 0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# 3. Cross Validation
lr_crossval = CrossValidator(estimator=lr_pipeline,
                             estimatorParamMaps=lr_paramGrid,
                             evaluator=RegressionEvaluator(labelCol="price", metricName="rmse"),
                             numFolds=3)

# 4. Train & Evaluate
lr_cv_model = lr_crossval.fit(train_data)
lr_predictions = lr_cv_model.transform(test_data)

# Đánh giá
eval_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
eval_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")

rmse_lr = eval_rmse.evaluate(lr_predictions)
r2_lr = eval_r2.evaluate(lr_predictions)
best_lr = lr_cv_model.bestModel.stages[-1]

print(f"✅ KẾT QUẢ LINEAR REGRESSION:")
print(f"   - R2 Score: {r2_lr:.4f}")
print(f"   - RMSE: ${rmse_lr:,.0f}")
print(f"   - Tham số tối ưu: regParam={best_lr.getRegParam()} | elasticNetParam={best_lr.getElasticNetParam()}")

🚀 [1/3] ĐANG CHẠY LINEAR REGRESSION (ELASTIC NET)...
   (Đang dò tìm tham số phạt regParam và elasticNetParam tốt nhất...)
✅ KẾT QUẢ LINEAR REGRESSION:
   - R2 Score: 0.6750
   - RMSE: $207,632
   - Tham số tối ưu: regParam=0.5 | elasticNetParam=1.0


# RANDOM FOREST

In [14]:
print("\nĐANG CHẠY RANDOM FOREST (TUNING)...")

# 1. Khởi tạo
rf = RandomForestRegressor(featuresCol="features", labelCol="price", seed=42)
rf_pipeline = Pipeline(stages=[assembler, rf])

# 2. Tạo lưới tham số (Grid Search)
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20, 50, 100]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

# 3. Cross Validation
rf_crossval = CrossValidator(estimator=rf_pipeline,
                             estimatorParamMaps=rf_paramGrid,
                             evaluator=eval_rmse,
                             numFolds=3)

# 4. Train & Evaluate
rf_cv_model = rf_crossval.fit(train_data)
rf_predictions = rf_cv_model.transform(test_data)

rmse_rf = eval_rmse.evaluate(rf_predictions)
r2_rf = eval_r2.evaluate(rf_predictions)
best_rf = rf_cv_model.bestModel.stages[-1]

print(f"✅ KẾT QUẢ RANDOM FOREST:")
print(f"   - R2 Score: {r2_rf:.4f}")
print(f"   - RMSE: ${rmse_rf:,.0f}")
print(f"   - Tham số tối ưu: NumTrees={best_rf.getNumTrees} | MaxDepth={best_rf.getOrDefault('maxDepth')}")


🚀ĐANG CHẠY RANDOM FOREST (TUNING)...
✅ KẾT QUẢ RANDOM FOREST:
   - R2 Score: 0.8659
   - RMSE: $133,353
   - Tham số tối ưu: NumTrees=100 | MaxDepth=15


# So sánh 2 mô hình



In [16]:

# Tạo DataFrame so sánh
comparison_data = [
    {
        "Model": "Linear Regression ",
        "R2 Score": r2_lr,
        "RMSE ($)": rmse_lr,
        "Ghi chú": "Mô hình cơ sở (Tuyến tính)"
    },
    {
        "Model": "Random Forest",
        "R2 Score": r2_rf,
        "RMSE ($)": rmse_rf,
        "Ghi chú": "Mô hình nâng cao (Phi tuyến tính)"
    }
]

df_compare = pd.DataFrame(comparison_data)

print("-" * 80)
print(df_compare.to_string(index=False))
print("-" * 80)

# Nhận xét tự động
diff_r2 = (r2_rf - r2_lr) * 100
print(f"💡 NHẬN XÉT: Random Forest cải thiện độ chính xác thêm {diff_r2:.2f}% so với Linear Regression.")

--------------------------------------------------------------------------------
             Model  R2 Score      RMSE ($)                           Ghi chú
Linear Regression   0.675018 207631.883070        Mô hình cơ sở (Tuyến tính)
     Random Forest  0.865947 133352.878974 Mô hình nâng cao (Phi tuyến tính)
--------------------------------------------------------------------------------
💡 NHẬN XÉT: Random Forest cải thiện độ chính xác thêm 19.09% so với Linear Regression.
